# Read original Titanic data and preprocess for training

In [ ]:
!pip install sklearn
!pip install hdfs

In [2]:
# linear algebra
import numpy as np 
# data processing
import pandas as pd 
# data storage
from hdfs import InsecureClient

In [3]:
client = InsecureClient('http://datalake:50070')
with client.read('/shared/Titanic/titanic_train_raw.csv', encoding = 'utf-8') as reader:
    train_df = pd.read_csv(reader)
with client.read('/shared/Titanic/titanic_test_raw.csv', encoding = 'utf-8') as reader:
    test_df = pd.read_csv(reader)

# train_df.info()
train_df.describe()

,pclass,age,sibsp,parch,fare
count,668.000000,539.000000,668.000000,668.000000,668.000000
mean,2.323353,29.537106,0.508982,0.387725,31.938678
std,0.837784,14.554157,1.103997,0.823729,48.989578
min,1.000000,0.420000,0.000000,0.000000,0.000000
25%,2.000000,20.000000,0.000000,0.000000,7.895800
50%,3.000000,28.000000,0.000000,0.000000,14.458300
75%,3.000000,39.000000,1.000000,0.000000,31.275000
max,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
total = train_df.isnull().sum().sort_values(ascending=False)
percent_1 = train_df.isnull().sum()/train_df.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data.head(5)

,Total,%
deck,519,77.7
age,129,19.3
embark_town,2,0.3
embarked,2,0.3
alone,0,0.0


In [5]:
train_df.columns.values

array(['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked',
       'class', 'who', 'adult_male', 'deck', 'embark_town', 'alive',
       'alone'], dtype=object)

## Preprocessing

In [6]:
train_df = train_df.drop(['embark_town'], axis=1)
train_df = train_df.drop(['parch'], axis=1)
train_df = train_df.drop(['deck'], axis=1)
train_df = train_df.drop(['class'], axis=1)
test_df = test_df.drop(['embark_town'], axis=1)
test_df = test_df.drop(['parch'], axis=1)
test_df = test_df.drop(['deck'], axis=1)
test_df = test_df.drop(['class'], axis=1)

data = [train_df, test_df]

## Converting Values

In [7]:
alive = {"yes": 1, "no": 0}
alone = {True: 1, False: 0}
adult_male = {True: 1, False: 0}
who = {"man": 1, "woman": 2, "child": 0}
for dataset in data:
  dataset['alive'] = dataset['alive'].map(alive)
  dataset['alone'] = dataset['alone'].map(alone)
  dataset['adult_male'] = dataset['adult_male'].map(alone)
  dataset['who'] = dataset['who'].map(who)

In [8]:
common_value = 'S'
for dataset in data:
    dataset['embarked'] = dataset['embarked'].fillna(common_value)
# train_df['embarked'].describe()

In [9]:
for dataset in data:
    dataset['age'] = dataset['age'].fillna(0)
    dataset['fare'] = dataset['fare'].fillna(0)
    dataset['fare'] = dataset['fare'].astype(int)

In [10]:
genders = {"male": 0, "female": 1}
for dataset in data:
    dataset['sex'] = dataset['sex'].map(genders)

In [11]:
ports = {"S": 0, "C": 1, "Q": 2}
data = [train_df, test_df]

for dataset in data:
    dataset['embarked'] = dataset['embarked'].map(ports)

In [12]:
data = [train_df, test_df]
for dataset in data:
    dataset['age'] = dataset['age'].astype(int)
    dataset.loc[ dataset['age'] <= 11, 'age'] = 0
    dataset.loc[(dataset['age'] > 11) & (dataset['age'] <= 18), 'age'] = 1
    dataset.loc[(dataset['age'] > 18) & (dataset['age'] <= 22), 'age'] = 2
    dataset.loc[(dataset['age'] > 22) & (dataset['age'] <= 27), 'age'] = 3
    dataset.loc[(dataset['age'] > 27) & (dataset['age'] <= 33), 'age'] = 4
    dataset.loc[(dataset['age'] > 33) & (dataset['age'] <= 40), 'age'] = 5
    dataset.loc[(dataset['age'] > 40) & (dataset['age'] <= 66), 'age'] = 6
    dataset.loc[ dataset['age'] > 66, 'age'] = 6

# let's see how it's distributed 
train_df['age'].value_counts()

0    182
6    113
4     93
3     78
5     74
2     71
1     57
Name: age, dtype: int64

In [13]:
for dataset in data:
    dataset.loc[ dataset['fare'] <= 7.91, 'fare'] = 0
    dataset.loc[(dataset['fare'] > 7.91) & (dataset['fare'] <= 14.454), 'fare'] = 1
    dataset.loc[(dataset['fare'] > 14.454) & (dataset['fare'] <= 31), 'fare']   = 2
    dataset.loc[(dataset['fare'] > 31) & (dataset['fare'] <= 99), 'fare']   = 3
    dataset.loc[(dataset['fare'] > 99) & (dataset['fare'] <= 250), 'fare']   = 4
    dataset.loc[ dataset['fare'] > 250, 'fare'] = 5
    dataset['fare'] = dataset['fare'].astype(int)

In [14]:
test_df.head(200)

,pclass,sex,age,sibsp,fare,embarked,who,adult_male,alive,alone
0,3,0,2,0,0,1,1,1,1,1
1,2,0,0,1,1,0,0,0,1,0
2,3,1,0,3,2,0,0,0,0,0
3,3,1,2,0,1,0,2,0,0,1
4,3,1,4,0,1,0,2,0,1,1
...,...,...,...,...,...,...,...,...,...,...
195,1,1,5,0,4,0,2,0,1,1
196,1,1,5,1,4,0,2,0,1,0
197,2,0,5,0,1,0,1,1,0,1
198,2,0,2,1,1,0,1,1,0,0


In [20]:
test_df = test_df.drop("alive", axis = 1)

In [21]:
test_df.head()

,pclass,sex,age,sibsp,fare,embarked,who,adult_male,alone
0,3,0,2,0,0,1,1,1,1
1,2,0,0,1,1,0,0,0,0
2,3,1,0,3,2,0,0,0,0
3,3,1,2,0,1,0,2,0,1
4,3,1,4,0,1,0,2,0,1


In [22]:
# save to file
train_df.to_csv(r'titanic_train.csv', index=None, header=True)
test_df.to_csv(r'titanic_test.csv', index=None, header=True)